Tensorflow Tutorial
===================

## 1. Graph Construction and Session
- Tensorflow는 기본적으로 값과 연산을 가지는 node들로 구성된 Directed Acyclic Graph를 형성하여 구현되는 구조를 가진다. 

### 1.1. tf.constant() & tf.Session()
- tf.constant()는 변하지 않은 값을 가지는 node를 생성한다.
- tf.constant(value, dtype=None, shape=None, name="Const', verify_shape=False)
- tf.sesstion()은 형성된 graph를 GPU/CPU에 올려 실행(계산)하는 역할을 한다.

#### 1.1.1. compute '3+4'

In [1]:
import tensorflow as tf

- 3과 4를 담는 node를 각각 생성

In [ ]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

- tf.Session 을 통해서 graph의 node를 run하면 node 값을 계산하고 결과를 return된다.

In [ ]:
sess = tf.Session()
print(sess.run([node1, node2]))

In [ ]:
node3 = tf.add(node1, node2)
print "node3: ",node3
print "sess.run(node3): ",sess.run(node3)


#### 1.1.2. Compute ‘((3+4)^2)*5’

In [ ]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0, tf.float32)
node3 = tf.add(node1, node2)
node4 = tf.constant(2.0, tf.float32)
node5 = tf.pow(node3, node4)
node6 = tf.constant(5.0, tf.float32)
node7 = tf.multiply(node5, node6)

print "3 + 4 = ",sess.run(node3)
print "(3 + 4)^2 = ",sess.run(node5)
print "((3 + 4)^2) * 5 = ",sess.run(node7)

### 1.2. tf.placeholder()
- tf.placeholder는 graph를 run할 때 값을 입력해주어야 하는 node이다.
- 함수의 input 즉 f(x)에서 x의 역할과도 같다.

#### 1.2.1. Compute 'x+y'
- tf.placeholder를 이용하여 sess.run할때마다 x,y에 input값을 주어 계산한다.

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
adder = x + y

z1 = sess.run(adder, {x: 3, y: 4.5})
print(z1)

z2 = sess.run(adder, {x: [1,3], y: [2, 4]})
print(z2)

#### 1.2.2. Compute '||Ax-y||'


In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
A = tf.constant([[1., 2.], [3., 4.]])
y_ = tf.matmul(A,x)
diff = y - y_
err = tf.norm(diff)

# should pass x to 2D matrix shape
print "A*x = \n",sess.run(y_, {x: [[1.], [2.]]})
print "|Ax-y| = \n",sess.run(err, {x: [[1.], [2.]], y:[[8.], [7.]]})

## 2. Variables
- tf.Variable()는 tf.constant()와 같이 값이 고정된 것이 아니고 값을 변경할 수 있는 node이다.
- 예를들어, f(x) = 3*x^2 + 2*x + 1 라 했을 때 x는 placeholder이고 3,2,1은 값을 변경할 상수라 하면 tf.constant이다.
- 그런데, f(x) = a*x^2 + b*x + c 에서 필요에 따라 a,b,c 값을 변경해줄 수 있다면, 이들은 tf.Variable로 정의한다.
- tf.Variable은 결국 function(graph)의 parameter이고, learning이 가능하다.
- tf.Variable(initial_value, name=optional_name)

### 2.1. Learning a Linear Model
- 이 chapter에서는 주어진 데이터로부터 함수 f(x)를 선형회귀하는 모델 Wx+b 를 learning한다. 
- Initializer는 텐서플로우 variable들의 정의된 초기값을 부여해준다. 
- tf.global_initializer는 graph 내의 정의된 모든 variable을 초기화시키는 연산자.
- 초기화 연산자를 sess.run()으로 실행시켜 초기화해준다. 

In [ ]:
W = tf.Variable([.3], tf.float32)  # Variable adds learnable parameters in graph
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

- sess.run에 계산할 node 이름과 거기에 필요한 placeholder 값을 주어 결과를 뱉는다.
- 하지만, variable들을 initialize하지 않고 graph를 run하면 에러 발생

In [ ]:
# without initializing variable, running graph causes error
print(sess.run(linear_model, {x:[1,2,3,4]}))

- global_variables_initializer로 graph에 정의된 모든 variable들을 초기화하고 다시 시도하면 성공.

In [ ]:
# run variable initializer op, before running the graph
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# Then, it works
print(sess.run(linear_model, {x:[1,2,3,4]}))

- loss 함수는 squared error function 사용

In [ ]:
# Define loss
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

print "loss : ",sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]})

- model을 training하기 전에, variable의 값 변경하는 법을 소개..
- tf.assign(variable, modified_value) 하여 variable의 값을 modified_value로 바꾸는 연산자 생성하고
- sess.run()에 넘겨 값을 바꾼다.

In [ ]:
# assign different value to the variables
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])   # These are optimal parameters

print "Before assigning, W : ",sess.run(W)[0], "b : ",sess.run(b)[0]
sess.run([fixW, fixb])
print "After assigning, W : ",sess.run(W)[0], "b : ",sess.run(b)[0]

print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

- Initializer을 다시 run하면 처음에 정의된 값으로 초기화.

In [ ]:
# Go back to the initial value
sess.run(init)
print "Ater initializing again, W : ",sess.run(W)[0], "b : ",sess.run(b)[0]

- tf.train에 정의된 optimizer중 하나를 이용하여 variable들을 learning한다.
- 예시에서는 일반적인 Stochastic Gradient Descent (SGD) optimizer 이용.
- Optimizer는 loss를 minimizing하는 방향으로 배우도록 한다.

In [ ]:
# Learn the model using optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

for i in range(1000):
    _, l = sess.run([train, loss], {x:[1,2,3,4], y:[0,-1,-2,-3]})
    if (i+1) % 100 == 0:
        print "iteration : ", i+1 ," loss : ", l
    
print "Learned model parameters, \t W : ",sess.run(W)[0],", b : ",sess.run(b)[0]
print "Optimal parameters are,   \t W :  -1 \t, b :  1"

### 2.2. Initializing Variables
- 여러가지 initialize 방법들 소개

#### 2.2.1. tf.global_variable_initializer()
- model의 모든 variable 초기화

In [ ]:
W1 = tf.Variable([.2], tf.float32)
W2 = tf.Variable([.4], tf.float32)
b = tf.Variable([-.3], tf.float32)

init = tf.global_variables_initializer()
sess.run(init)
print "W1 :",sess.run(W1)[0],",\t W2 :",sess.run(W2)[0],",\t b :",sess.run(b)[0]

#### 2.2.2 tf.variables_initialize()
- tf.variables_initialize(var_list)는 var_list (variables들의 list)에 포함된 variables들만 initialize

In [ ]:
# 위에 정의된 variable들의 값을 바꾼다.
assn_W1 = W1.assign([.1])
assn_W2 = W2.assign([.1])
assn_b = b.assign([-.1])

sess.run([assn_W1, assn_W2, assn_b])
print "W1 :",sess.run(W1)[0],",\t W2 :",sess.run(W2)[0],",\t b :",sess.run(b)[0]

In [ ]:
# W1, b만 초기값으로 initialize한다.
init_W = tf.variables_initializer([W1, b])
# init_W = tf.variables_initializer(set(tf.global_variables())-W2)
sess.run(init_W)
print "W1 :",sess.run(W1)[0],",\t W2 :",sess.run(W2)[0],",\t b :",sess.run(b)[0]

#### 2.2.3 Variable.initialized_value()
- 두 변수 A, B를 동일한 값으로 초기화시킬 때, Variable의 initialized_value()를 이용한다.
- initialized_value는 그 변수의 초기화 값을 return한다.

In [ ]:
A = tf.Variable(tf.random_normal([2,3], stddev=0.35), name="A")
# A와 같은 초기 값을 가지는 변수 B를 만든다.
B = tf.Variable(A.initialized_value(), name="B")
# A의 초기값의 정확히 2배의 초기값을 가지는 변수 C를 만든다.
C = tf.Variable(A.initialized_value() * 2.0, name="C")

init_ABC = tf.variables_initializer(set([A,B,C]))
sess.run(init_ABC)
print "A :\n",sess.run(A),"\nB :\n",sess.run(B),"\nC :\n",sess.run(C)

### 2.4 Saving and Loading Variables
- 배운 모델을 저장하거나, 배워진 모델을 읽어들일 때 필요한 과정.
- tf.train.Saver()을 사용한다.
- 모든 변수가 아니라, 지정한 몇 개의 변수만 저장하고 불러올 수도 있다.
- key : 저장/불러올 변수 이름의 이름, value : 저장/불러올 변수 를 가지는 python dictionary를 만들어서 train.Saver의 input으로 넘긴다.
- train.Saver의 input이 없을 때는 default로 graph 내의 모든 변수들을 save/restore한다. 
- graph 내의 모든 변수들을 보려면 global_variables() 함수를 사용하여, print(global_bvariables()) 등을 이용.

#### 2.4.1 Save model : all variables
- tf.train.Saver.save()를 사용.

In [ ]:
# Variable 생성
v1 = tf.Variable(tf.random_normal([2,2]), name="v1")
v2 = tf.Variable(tf.random_normal([2,1]), name="v2")
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

init_op = tf.global_variables_initializer()

# tf.train.Saver() 을 먼저 생성. 저장할 변수들 pass 
saver = tf.train.Saver({"w1": v1, "w2": v2})
# saver = tf.train.Saver([v1, v2])   # name은 default(v.op.name)으로 들어감
# saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})

# 모델 생성, 변수 초기화, training, 모델 저장.
with tf.Session() as sess:
    # model 생성
    loss = tf.norm(tf.matmul(v1,x)+v2-y)
    
    # 변수 초기화
    sess.run(init_op)
    
    # training
    optimizer = tf.train.GradientDescentOptimizer(0.001)
    train = optimizer.minimize(loss)
    
    for i in range(200):
        _, l = sess.run([train, loss], {x:[[1.,2.,3.,4.],[0.,1.,2.,3.]], y:[[0.,-1.,-2.,-3.],[-1.,-2.,-3.,-4.]]})
        if (i+1) % 20 == 0:
            print "iteration : ", i+1 ," loss : ", l
    
    # 디스크에 변수 저장
    save_path = saver.save(sess, "model.ckpt")  # 세션과 저장 파일 이름 지정
    print "v1 :\n",sess.run(v1), "\nv2 :\n", sess.run(v2)
    print("Model (variables v1, v2) saved in file: %s" % save_path)

#### 2.4.2 Load model : all variables
- 변수들을 저장된 값을 불러 초기화한다. 
- 이 경우 restore된 변수들은 따로 initializer로 초기화할 필요 없다.
- tf.train.Saver.restore()를 사용
- 밑의 예시는 위 예시에서 저장된 checkpoint부터 시작하여 training을 이어나가는 과정.

In [ ]:
# Variable 생성
w1 = tf.Variable(tf.random_normal([2,2]), name="w1")
w2 = tf.Variable(tf.random_normal([2,1]), name="w2")
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# restorer를 정의하여 restore할 
restorer = tf.train.Saver({"w1": w1, "w2": w2})
with tf.Session() as sess:
    # model 생성
    loss = tf.norm(tf.matmul(w1,x)+w2-y)
    
    # checkpoint 파일로부터 변수값 읽어와서 변수 초기화
    restorer.restore(sess, "model.ckpt")
    print("Model restored.")
    print "w1 :\n",sess.run(w1), "\nw2 :\n", sess.run(w2)
    
    # training
    optimizer = tf.train.GradientDescentOptimizer(0.001)
    train = optimizer.minimize(loss)
    
    for i in range(200):
        _, l = sess.run([train, loss], {x:[[1.,2.,3.,4.],[0.,1.,2.,3.]], y:[[0.,-1.,-2.,-3.],[-1.,-2.,-3.,-4.]]})
        if (i+1) % 20 == 0:
            print "iteration : ", i+1 ," loss : ", l
    

- restore되지 않은 변수는 꼭 초기화를 해준다.화

In [ ]:
w3 = tf.Variable(tf.random_normal([2,1]), name="w3")
w4 = tf.Variable(tf.random_normal([2,1]), name="w4")

with tf.Session() as sess:
    restorer.restore(sess, "model.ckpt")
    print("Initialize w1, w2 by restoring saved values")

    # restore 되지 않은 w3는 초기화
    init_34 = tf.variables_initializer([w3, w4])
    sess.run(init_34)
    
    print "w1 : ",sess.run(w1), "\nw2 : ", sess.run(w2)
    print "\nw3 : ",sess.run(w3), "\nw4 : ", sess.run(w4)


## 3. Sharing Variables
- Variable은 선언할 때 마다 새로 생성되어 덮어씌어진다는 특징이 있는데, 이미 만든 variable은 새로 만들지 않고 기존의 것을 재사용하게 하도록 하는 방법이 필요할 때가 있다.
- 밑의 예시는 convolution layer를 포함한 model인데, 두가지의 문제점이 있다.
    - 첫째로, 함수 내의 모델이 고정되어있어 모델의 layer를 추가하려면 새로운함수를 만들거나, main함수에 직접 넣어주어야 하는 비효율이 발생한다.
    - 둘째로, 모델에 input을 넣어줄 때마다 새로운 variable이 생성되어, input에 따라 다른 filter를 통과하게.

In [ ]:
def my_image_filter(input_images):
    conv1_weights = tf.Variable(tf.random_normal([5, 5, 3, 32]), name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv1 = tf.nn.conv2d(input_images, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + conv1_biases)

    conv2_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]), name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    conv2 = tf.nn.conv2d(relu1, conv2_weights,  strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + conv2_biases)


In [2]:
from scipy import misc

image1 = tf.expand_dims(tf.constant(misc.imread('acoustic-guitar-player.jpg'), tf.float32), 0)
image2 = tf.expand_dims(tf.constant(misc.imread('iris.jpg'), tf.float32), 0)

- 위에서 언급한 두 번째 문제 발생

In [ ]:
temp = tf.global_variables()
# First call creates one set of 4 variables.
result1 = my_image_filter(image1)
print "Number of created variables : \n",len(set(tf.global_variables())-set(temp))

# Another set of 4 variables is created in the second call.
result2 = my_image_filter(image2)
print "Number of created variables : \n",len(set(tf.global_variables())-set(temp))

- tf.placeholder를 이용하여 sess.run 할때마다 input을 넘겨주거나, variable들을 dictionary로 따로 함수 밖에 만들어놓은 뒤 함수에서 refer하면 두 번째 문제는 해결되지만, 첫번째 문제는 여전히 해결하지 못한다.
- tf.variable_scope 와 tf.get_variable을 이용하여 두 문제를 효율적으로 해결해보자.

### 3.1. tf.get_variable() & tf.variable_scope()


In [ ]:
v = tf.get_variable("v", [1])
v1 = tf.get_variable("v", [1])

In [ ]:
with tf.variable_scope("foo"):                  # tf.get_variable_scope().reuse == False
    v = tf.get_variable("v", [1])
    tf.get_variable_scope().reuse_variables()   # tf.get_variable_scope().reuse == True	
    v1 = tf.get_variable("v", [1])
assert v1 is v

In [2]:
with tf.variable_scope("fooo"):
    v = tf.get_variable("v", [1])
print(v.name)


with tf.variable_scope("fooo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
print(v.name)


with tf.variable_scope("gooo"):
    v = tf.get_variable("v", [1])
with tf.variable_scope("gooo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 is v     # v1 = v.

fooo/v:0
fooo/bar/v:0


In [3]:
with tf.variable_scope("root"):
    # At start, the scope is not reusing.
    print "reuse root ?   ", tf.get_variable_scope().reuse
    with tf.variable_scope("foo"):
        # Opened a sub-scope, still not reusing.
        print "reuse root/foo ?   ", tf.get_variable_scope().reuse
    with tf.variable_scope("foo", reuse=True):
        # Explicitly opened a reusing scope.
        print "reuse root/foo ?   ", tf.get_variable_scope().reuse
        with tf.variable_scope("bar"):
            # Now sub-scope inherits the reuse flag.
            print "reuse root/foo/bar ?   ", tf.get_variable_scope().reuse
    # Exited the reusing scope, back to a non-reusing one.
    print "reuse root ?   ", tf.get_variable_scope().reuse

reuse root ?    False
reuse root/foo ?    False
reuse root/foo ?    True
reuse root/foo/bar ?    True
reuse root ?    False


In [4]:
with tf.variable_scope("Foo") as foo_scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope(foo_scope):
    w = tf.get_variable("w", [1])
with tf.variable_scope(foo_scope, reuse=True):
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])
assert v1 is v
assert w1 is w

In [6]:
with tf.variable_scope("Goo") as foo_scope:
    assert foo_scope.name == "Goo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "Goo"  # Not changed.


In [8]:
with tf.variable_scope("hoo", initializer=tf.constant_initializer(0.4)):
    v = tf.get_variable("v", [1])
    print("hoo/v:0 : %f" % v.eval())  # Default initializer as set above.
    w = tf.get_variable("w", [1], initializer=tf.constant_initializer(0.3))
    print("hoo/w:0 : %f" % w.eval())  # Specific initializer overrides the default.
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        print("hoo/bar/v:0 : %f" % v.eval())  # Inherited default initializer.
    with tf.variable_scope("baz", initializer=tf.random_normal_initializer()):
        v = tf.get_variable("v", [1])
        print("hoo/baz/v:0 : %f" % v.eval())  # Changed default initializer.

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

### 3.2 Solving the previous problem

In [3]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights", “biases”.
    weights = tf.get_variable("weights", kernel_shape, 	initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 3, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])


In [4]:
with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(image1)
    scope.reuse_variables()
    result2 = my_image_filter(image2)

ValueError: Variable conv1/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-3-14037045eee3>", line 3, in conv_relu
    weights = tf.get_variable("weights", kernel_shape, 	initializer=tf.random_normal_initializer())
  File "<ipython-input-3-14037045eee3>", line 11, in my_image_filter
    relu1 = conv_relu(input_images, [5, 5, 3, 32], [32])
  File "<ipython-input-4-1fb90c781401>", line 1, in <module>
    result1 = my_image_filter(image1)


### 3.3. Name Scope

In [17]:
with tf.variable_scope("noo"):
    x = 1.0 + tf.get_variable("v", [1])
print(x.op.name)

noo/add


In [18]:
with tf.variable_scope("moo"):
    	with tf.name_scope("bar"):
        		v = tf.get_variable("v", [1])
        		x = 1.0 + v
print("v name : %s" % v.name)
print("x operation name : %s" % x.op.name)

v name : moo/v:0
x operation name : moo/bar/add
